In [1]:
import plotly.graph_objects as go
import plotly.express as px
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import shapely
import time
import random
from shapely import wkt
from shapely.ops import snap
from shapely.geometry.point import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
import pyproj
from plotly.graph_objs import Layout
from matplotlib.pyplot import plot

from string import digits
remove_digits = str.maketrans('', '', digits)

from rosreestr2coord import Area

from tqdm import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [3]:
def geoseries2shapedict(gdf, name_col, gdf_type):
    regions = {}
    for und, reg in tqdm(gdf.iterrows()):
        if type(reg.geometry) == MultiPolygon:
            x, y = np.array([[], []])
            for poly in reg.geometry.geoms:
                x_, y_ = poly.exterior.coords.xy
                x, y = (np.append(x, x_), np.append(y, y_))
                x, y = (np.append(x, None), np.append(y, None))
            x = x[:len(x)-1]
            y = y[:len(y)-1]
        elif type(reg.geometry) == Point:
            x, y = np.array(reg.geometry.coords.xy)
        else:
            x, y = np.array(reg.geometry.exterior.coords.xy)
        
        if gdf_type == 'rumap':
            regions[reg[name_col]] = x, y, reg.federal_district
        elif gdf_type == 'flood':
            regions[reg[name_col]] = x, y, reg.level
        elif gdf_type == 'regmap':
            regions[reg[name_col]] = x, y, reg.index
        else:
            regions[reg[name_col]] = x, y, reg.kvartal_cn
    return regions              

# load ru map

In [4]:
ru_map = gpd.read_file('map_data/russia_regions_stats.geojson')
ru_map = ru_map.to_crs('ESRI:102027')

In [5]:
ru_map

,name_ru,name_en,federal_district,count_osm,count_mapbox,mapbox_img_2019_2020_pct,ratio_mapbox_osm,updated,geometry
0,Пензенская область,Penza oblast,Приволжский,255374,300161,34.255814,1.175380,NaN,"MULTIPOLYGON (((-3034722.148 3368565.973, -303..."
1,Саратовская область,Saratov oblast,Приволжский,491455,601512,41.771953,1.223940,NaN,"MULTIPOLYGON (((-3136095.833 3178595.472, -313..."
2,Мордовия,Mordovia,Приволжский,191140,191942,81.121859,1.004200,NaN,"MULTIPOLYGON (((-3000292.487 3408729.339, -300..."
3,Тамбовская область,Tambov oblast,Центральный,240795,184504,58.284703,0.766229,NaN,"MULTIPOLYGON (((-3205780.449 3360578.790, -320..."
4,Рязанская область,Ryazan Oblast,Центральный,135783,238589,87.324990,1.757130,NaN,"MULTIPOLYGON (((-3170823.903 3534078.608, -317..."
...,...,...,...,...,...,...,...,...,...
80,Ивановская область,Ivanovo oblast,Центральный,88559,194870,88.912088,2.200450,NaN,"MULTIPOLYGON (((-2976095.236 3720906.910, -297..."
81,Костромская область,Kostroma oblast,Центральный,77873,135149,43.502731,1.735510,NaN,"MULTIPOLYGON (((-2893325.672 3734902.940, -289..."
82,Марий Эл,Mari El,Приволжский,152951,183194,92.948044,1.197730,NaN,"MULTIPOLYGON (((-2699463.030 3488398.120, -269..."
83,Магаданская область,Magadan oblast,Дальневосточный,9940,13626,47.188903,1.370820,NaN,"MULTIPOLYGON (((2997902.542 4430646.893, 29979..."


In [6]:
ru_map['area'] = ru_map.geometry.apply(lambda x: x.area)
ru_map_regions = geoseries2shapedict(ru_map.sort_values(by=['area'], axis=0, ascending=False), 'name_ru', 'rumap')

85it [00:00, 174.54it/s]


# oblasti regionov

In [7]:
a = pd.read_csv('map_data/kad_map/admin_level_6.csv', delimiter=';', encoding='utf8')

In [8]:
a = a.dropna(axis=1,thresh=len(a)*0.19)

In [9]:
a.iloc[a[a.name.apply(lambda x: 'ородской' in x)].index, 1] = a[a.name.apply(lambda x: 'ородской' in x)].name.apply(lambda x: x.replace('городской округ ', '').replace(' городской округ', ''))

In [10]:
gdf_regions = gpd.GeoDataFrame(a, geometry=wkt.loads(a.WKT), crs='EPSG:4326')

In [11]:
all_ru_regions = gdf_regions[['name', 'name__ru', 'wikipedia', 'geometry']]

In [12]:
all_ru_regions = all_ru_regions.to_crs('ESRI:102027')

In [13]:
all_ru_regions

,name,name__ru,wikipedia,geometry
0,Селенгинский район,Селенгинский район,ru:Селенгинский район,"MULTIPOLYGON (((772911.849 2183926.259, 770829..."
1,Хоринский район,Хоринский район,ru:Хоринский район,"MULTIPOLYGON (((975660.756 2392376.746, 968958..."
2,Курумканский район,Курумканский район,ru:Курумканский район,"MULTIPOLYGON (((1006551.636 2652187.027, 10017..."
3,Брюховецкий район,Брюховецкий район,ru:Брюховецкий район,"MULTIPOLYGON (((-3642267.062 2820599.306, -364..."
4,Павловский район,Павловский район,ru:Павловский район (Краснодарский край),"MULTIPOLYGON (((-3583010.429 2801858.794, -358..."
...,...,...,...,...
2362,Тамбов,NaN,ru:Тамбов,"MULTIPOLYGON (((-3124307.505 3303042.861, -312..."
2363,Рассказовский район,NaN,ru:Рассказовский район,"MULTIPOLYGON (((-3081265.438 3272682.587, -308..."
2364,Сосновский район,NaN,ru:Сосновский район (Тамбовская область),"MULTIPOLYGON (((-3079943.869 3330265.964, -308..."
2365,Тамбовский район,NaN,ru:Тамбовский район (Тамбовская область),"MULTIPOLYGON (((-3101331.947 3297711.561, -310..."


In [14]:
all_ru_regions_exploded = all_ru_regions.explode()

In [15]:
all_ru_regions_exploded.reset_index(drop=True, inplace=True)

In [16]:
all_ru_regions_exploded['area'] = all_ru_regions_exploded.geometry.apply(lambda x: x.area)

In [17]:
all_ru_regions_exploded = all_ru_regions_exploded.sort_values('area', ascending=False).drop_duplicates(['name', 'wikipedia']).sort_index()
all_ru_regions_exploded.reset_index(drop=True, inplace=True)

In [18]:
all_ru_regions_exploded

,name,name__ru,wikipedia,geometry,area
0,Селенгинский район,Селенгинский район,ru:Селенгинский район,"POLYGON ((772911.849 2183926.259, 770829.137 2...",6.729938e+09
1,Хоринский район,Хоринский район,ru:Хоринский район,"POLYGON ((975660.756 2392376.746, 968958.946 2...",1.216154e+10
2,Курумканский район,Курумканский район,ru:Курумканский район,"POLYGON ((1006551.636 2652187.027, 1001762.061...",1.117060e+10
3,Брюховецкий район,Брюховецкий район,ru:Брюховецкий район,"POLYGON ((-3642267.062 2820599.306, -3642492.8...",1.132083e+09
4,Павловский район,Павловский район,ru:Павловский район (Краснодарский край),"POLYGON ((-3583010.429 2801858.794, -3583399.7...",1.472633e+09
...,...,...,...,...,...
2360,Тамбов,NaN,ru:Тамбов,"POLYGON ((-3124307.505 3303042.861, -3124603.5...",7.601746e+07
2361,Рассказовский район,NaN,ru:Рассказовский район,"POLYGON ((-3081265.438 3272682.587, -3081149.2...",1.554211e+09
2362,Сосновский район,NaN,ru:Сосновский район (Тамбовская область),"POLYGON ((-3079943.869 3330265.964, -3082502.7...",2.043358e+09
2363,Тамбовский район,NaN,ru:Тамбовский район (Тамбовская область),"POLYGON ((-3101331.947 3297711.561, -3101893.6...",2.251141e+09


# таблица с кадастровыми кодами российских регионов

In [19]:
tables = pd.read_html("flood_proj/Кадастровый рейтинг регионов от ЕГРП365.html")

In [20]:
tab = tables[0]

In [21]:
tab.columns = [''] * len(tab.columns)

In [22]:
all_kads = tab.iloc[:,2][:-1]

In [23]:
all_kads_df = all_kads.apply(lambda x: x.replace(' кадастровый', '')).to_frame()

In [24]:
all_kads_df.columns = ['src']

In [25]:
all_regs_df = all_kads_df[all_kads_df.src.apply(lambda x: len(x.split('[')[1].split(':'))) != 2]

In [26]:
all_regs_df[['name', 'reg']] = [None, None]

In [27]:
all_regs_df.name = all_regs_df.src.apply(lambda x: x.split('[')[0][:-1])
all_regs_df.reg = all_regs_df.src.apply(lambda x: x.split('[')[1].split(':')[0][:-1])

In [28]:
all_regs_df.drop(columns=['src'], inplace=True)
all_regs_df.reset_index(drop=True, inplace=True)

In [29]:
all_kads_df = all_kads_df[all_kads_df.src.apply(lambda x: len(x.split('[')[1].split(':'))) == 2]

In [30]:
all_kads_df.columns = ['src']

In [31]:
all_kads_df[['name', 'reg', 'dst']] = [None, None, None]

In [32]:
all_kads_df.name = all_kads_df.src.apply(lambda x: x.split('[')[0][:-1])
all_kads_df.reg = all_kads_df.src.apply(lambda x: x.split('[')[1].split(':')[0])
all_kads_df.dst = all_kads_df.src.apply(lambda x: x.split('[')[1].split(':')[1][:-1])

In [33]:
all_kads_df.drop(columns=['src'], inplace=True)
all_kads_df.reset_index(drop=True, inplace=True)

In [34]:
all_kads_df.iloc[all_kads_df[all_kads_df.name.apply(lambda x: 'г. ' in x)].index, 0] = all_kads_df[all_kads_df.name.apply(lambda x: 'г. ' in x)].name.apply(lambda all_kads_df: all_kads_df.replace('г. ', ''))

In [35]:
all_kads_df

,name,reg,dst
0,Облученский район,79,05
1,Биробиджанский район,79,04
2,Октябрьский район,79,02
3,Смидовичский район,79,06
4,Биробиджан,79,01
...,...,...,...
2617,Гавриловский район,68,02
2618,Жердевский район,68,03
2619,Староюрьевский район,68,19
2620,Ржаксинский район,68,16


In [36]:
all_kads_df.reg = all_kads_df.reg.astype(int)
all_kads_df.dst = all_kads_df.dst.astype(int)

In [37]:
all_ru_regions['dst'] = None

In [38]:
all_ru_regions

,name,name__ru,wikipedia,geometry,dst
0,Селенгинский район,Селенгинский район,ru:Селенгинский район,"MULTIPOLYGON (((772911.849 2183926.259, 770829...",None
1,Хоринский район,Хоринский район,ru:Хоринский район,"MULTIPOLYGON (((975660.756 2392376.746, 968958...",None
2,Курумканский район,Курумканский район,ru:Курумканский район,"MULTIPOLYGON (((1006551.636 2652187.027, 10017...",None
3,Брюховецкий район,Брюховецкий район,ru:Брюховецкий район,"MULTIPOLYGON (((-3642267.062 2820599.306, -364...",None
4,Павловский район,Павловский район,ru:Павловский район (Краснодарский край),"MULTIPOLYGON (((-3583010.429 2801858.794, -358...",None
...,...,...,...,...,...
2362,Тамбов,NaN,ru:Тамбов,"MULTIPOLYGON (((-3124307.505 3303042.861, -312...",None
2363,Рассказовский район,NaN,ru:Рассказовский район,"MULTIPOLYGON (((-3081265.438 3272682.587, -308...",None
2364,Сосновский район,NaN,ru:Сосновский район (Тамбовская область),"MULTIPOLYGON (((-3079943.869 3330265.964, -308...",None
2365,Тамбовский район,NaN,ru:Тамбовский район (Тамбовская область),"MULTIPOLYGON (((-3101331.947 3297711.561, -310...",None


In [39]:
all_kads_df

,name,reg,dst
0,Облученский район,79,5
1,Биробиджанский район,79,4
2,Октябрьский район,79,2
3,Смидовичский район,79,6
4,Биробиджан,79,1
...,...,...,...
2617,Гавриловский район,68,2
2618,Жердевский район,68,3
2619,Староюрьевский район,68,19
2620,Ржаксинский район,68,16


# соединение кадастровых номеров с картой 

# ru map convert

In [40]:
map_name = ru_map['name_ru'].unique().tolist()

In [41]:
list_name = all_regs_df['name'].unique().tolist()

In [42]:
list(set(map_name) - set(list_name))

['Еврейская автономная область',
 'Ямало-Ненецкий автономный округ',
 'Республика Алтай',
 'Ханты-Мансийский автономный округ']

In [43]:
list(set(list_name) - set(map_name))

['Ямало-Ненецкий АО', 'Алтай', 'Ханты-Мансийский АО']

In [44]:
# переименовал небьющиеся регионы
for i in list(set(map_name) - set(list_name)):
    print(i)
    ru_map['temp_bool'] = ru_map.name_ru.apply(lambda x: x == i)
    print('введи замену')
    asd = input()
    ru_map.iloc[ru_map[ru_map['temp_bool']]['name_ru'].index, 0] = asd

Еврейская автономная область
введи замену
Еврейская АО
Ямало-Ненецкий автономный округ
введи замену
Ямало-Ненецкий АО
Республика Алтай
введи замену
Алтай
Ханты-Мансийский автономный округ
введи замену
Ханты-Мансийский АО


In [45]:
ru_map.drop(columns=['temp_bool'], inplace=True)

In [46]:
ru_map_reg = ru_map.set_index('name_ru').join(all_regs_df.set_index('name')).reset_index()

In [47]:
ru_map_reg[ru_map_reg.reg.isna()]

,name_ru,name_en,federal_district,count_osm,count_mapbox,mapbox_img_2019_2020_pct,ratio_mapbox_osm,updated,geometry,area,reg
34,Еврейская АО,Jewish autonomous oblast,Дальневосточный,16702,21981,16.792118,1.31607,NaN,"MULTIPOLYGON (((2320683.203 2366197.542, 23212...",2.971101e+10,NaN


In [48]:
ru_map_reg.iloc[34,-1] = 79

In [49]:
ru_map_reg.to_csv('map_data/ru_regions_new.csv', index=None)

In [50]:
ru_map_reg = pd.read_csv('map_data/ru_regions_new.csv')

In [51]:
ru_map_reg

,name_ru,name_en,federal_district,count_osm,count_mapbox,mapbox_img_2019_2020_pct,ratio_mapbox_osm,updated,geometry,area,reg
0,Пензенская область,Penza oblast,Приволжский,255374,300161,34.255814,1.175380,NaN,MULTIPOLYGON (((-3034722.148410227 3368565.972...,3.706563e+10,58
1,Саратовская область,Saratov oblast,Приволжский,491455,601512,41.771953,1.223940,NaN,MULTIPOLYGON (((-3136095.832603514 3178595.471...,8.545637e+10,64
2,Мордовия,Mordovia,Приволжский,191140,191942,81.121859,1.004200,NaN,MULTIPOLYGON (((-3000292.4874718864 3408729.33...,2.256968e+10,13
3,Тамбовская область,Tambov oblast,Центральный,240795,184504,58.284703,0.766229,NaN,MULTIPOLYGON (((-3205780.449216651 3360578.790...,2.933953e+10,68
4,Рязанская область,Ryazan Oblast,Центральный,135783,238589,87.324990,1.757130,NaN,MULTIPOLYGON (((-3170823.903206058 3534078.607...,3.427317e+10,62
...,...,...,...,...,...,...,...,...,...,...,...
80,Ивановская область,Ivanovo oblast,Центральный,88559,194870,88.912088,2.200450,NaN,MULTIPOLYGON (((-2976095.236404902 3720906.910...,1.918202e+10,37
81,Костромская область,Kostroma oblast,Центральный,77873,135149,43.502731,1.735510,NaN,MULTIPOLYGON (((-2893325.6717919535 3734902.94...,5.430535e+10,44
82,Марий Эл,Mari El,Приволжский,152951,183194,92.948044,1.197730,NaN,MULTIPOLYGON (((-2699463.0299782963 3488398.11...,2.062989e+10,12
83,Магаданская область,Magadan oblast,Дальневосточный,9940,13626,47.188903,1.370820,NaN,MULTIPOLYGON (((2997902.5421598535 4430646.893...,4.436866e+11,49


In [52]:
ru_map_reg = gpd.GeoDataFrame(ru_map_reg, geometry=wkt.loads(ru_map_reg.geometry), crs='ESRI:102027')

In [53]:
ru_map_reg

,name_ru,name_en,federal_district,count_osm,count_mapbox,mapbox_img_2019_2020_pct,ratio_mapbox_osm,updated,geometry,area,reg
0,Пензенская область,Penza oblast,Приволжский,255374,300161,34.255814,1.175380,NaN,"MULTIPOLYGON (((-3034722.148 3368565.973, -303...",3.706563e+10,58
1,Саратовская область,Saratov oblast,Приволжский,491455,601512,41.771953,1.223940,NaN,"MULTIPOLYGON (((-3136095.833 3178595.472, -313...",8.545637e+10,64
2,Мордовия,Mordovia,Приволжский,191140,191942,81.121859,1.004200,NaN,"MULTIPOLYGON (((-3000292.487 3408729.339, -300...",2.256968e+10,13
3,Тамбовская область,Tambov oblast,Центральный,240795,184504,58.284703,0.766229,NaN,"MULTIPOLYGON (((-3205780.449 3360578.790, -320...",2.933953e+10,68
4,Рязанская область,Ryazan Oblast,Центральный,135783,238589,87.324990,1.757130,NaN,"MULTIPOLYGON (((-3170823.903 3534078.608, -317...",3.427317e+10,62
...,...,...,...,...,...,...,...,...,...,...,...
80,Ивановская область,Ivanovo oblast,Центральный,88559,194870,88.912088,2.200450,NaN,"MULTIPOLYGON (((-2976095.236 3720906.910, -297...",1.918202e+10,37
81,Костромская область,Kostroma oblast,Центральный,77873,135149,43.502731,1.735510,NaN,"MULTIPOLYGON (((-2893325.672 3734902.940, -289...",5.430535e+10,44
82,Марий Эл,Mari El,Приволжский,152951,183194,92.948044,1.197730,NaN,"MULTIPOLYGON (((-2699463.030 3488398.120, -269...",2.062989e+10,12
83,Магаданская область,Magadan oblast,Дальневосточный,9940,13626,47.188903,1.370820,NaN,"MULTIPOLYGON (((2997902.542 4430646.893, 29979...",4.436866e+11,49


## соединение

In [54]:
all_ru_regions = all_ru_regions_exploded.copy()

In [55]:
all_ru_regions['reg'] = None

In [56]:
all_ru_regions.geometry = all_ru_regions.geometry.scale(xfact=0.5, yfact=0.5, origin='center')

In [57]:
for i, all_val in all_ru_regions.iterrows():
    cnter = 0
    for j, ru_val in ru_map_reg.iterrows():
        try:
            if ru_val.geometry.contains(all_val.geometry):
                all_ru_regions.iloc[i,-1] = ru_val.reg
                cnter += 1
            else:
                pass
        except:
            print(i, j, 'error')
    if cnter > 1:
        print(i, j, cnter)
        print()

In [58]:
all_ru_regions

,name,name__ru,wikipedia,geometry,area,reg
0,Селенгинский район,Селенгинский район,ru:Селенгинский район,"POLYGON ((744938.253 2182827.287, 743896.897 2...",6.729938e+09,3
1,Хоринский район,Хоринский район,ru:Хоринский район,"POLYGON ((941875.394 2369781.309, 938524.489 2...",1.216154e+10,3
2,Курумканский район,Курумканский район,ru:Курумканский район,"POLYGON ((979970.143 2625112.487, 977575.355 2...",1.117060e+10,3
3,Брюховецкий район,Брюховецкий район,ru:Брюховецкий район,"POLYGON ((-3656390.883 2829605.816, -3656503.7...",1.132083e+09,23
4,Павловский район,Павловский район,ru:Павловский район (Краснодарский край),"POLYGON ((-3595906.609 2812435.065, -3596101.2...",1.472633e+09,23
...,...,...,...,...,...,...
2360,Тамбов,NaN,ru:Тамбов,"POLYGON ((-3126682.555 3305113.619, -3126830.5...",7.601746e+07,68
2361,Рассказовский район,NaN,ru:Рассказовский район,"POLYGON ((-3094356.417 3273058.895, -3094298.3...",1.554211e+09,68
2362,Сосновский район,NaN,ru:Сосновский район (Тамбовская область),"POLYGON ((-3093953.800 3342204.861, -3095233.2...",2.043358e+09,68
2363,Тамбовский район,NaN,ru:Тамбовский район (Тамбовская область),"POLYGON ((-3119140.653 3304911.081, -3119421.4...",2.251141e+09,68


In [59]:
all_ru_regions.geometry = all_ru_regions.geometry.scale(xfact=2, yfact=2, origin='center')

In [60]:
all_ru_regions['dst'] = None

In [61]:
for i, all_val in all_ru_regions.iterrows():
    cnter = 0
    for j, kad_val in all_kads_df[all_kads_df.reg == all_val.reg].iterrows():
        if all_val['name'] == kad_val['name'] \
        or all_val['name'].replace('ё', 'е') == kad_val['name'] \
        or all_val['name'].replace('улус', 'район').replace('национальный улус', 'район') == kad_val['name'] \
        or all_val['name'][1:-1] == kad_val['name'][1:-1] \
        or all_val['name'].replace('Зато ', '').replace('ЗАТО ', '') == kad_val['name'].replace(' район', ''):
            all_ru_regions.iloc[i, -1] = kad_val['dst']
            cnter += 1
        else:
            pass
    if cnter > 2:
        print(cnter, i)

In [62]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [63]:
for i, all_val in all_ru_regions[all_ru_regions.dst.isna()].iterrows():
    cnter = 0
    for j, kad_val in all_kads_df[all_kads_df.reg == all_val.reg].iterrows():
        if all_val['name'].replace('Зато', '').replace('ЗАТО', '').replace('район', '').replace('национальный улус', '').replace('Национальный улус', '').replace('улус', 'улус').replace('ё', 'е').replace('Ё', 'Е').replace('ий', '').replace('ый', '').replace(' ', '').lower() ==\
        kad_val['name'].replace('Зато', '').replace('ЗАТО', '').replace('район', '').replace('национальный улус', '').replace('Национальный улус', '').replace('улус', 'улус').replace('ё', 'е').replace('Ё', 'Е').replace('ий', '').replace('ый', '').replace(' ', '').lower() \
        or similar(all_val['name'].replace('Зато', '').replace('ЗАТО', '').replace('район', '').replace('национальный улус', '').replace('Национальный улус', '').replace('улус', 'улус').replace('ё', 'е').replace('Ё', 'Е').replace('ий', '').replace('ый', '').replace(' ', '')[1:], kad_val['name'].replace('Зато', '').replace('ЗАТО', '').replace('район', '').replace('национальный улус', '').replace('Национальный улус', '').replace('улус', 'улус').replace('ё', 'е').replace('Ё', 'Е').replace('ий', '').replace('ый', '').replace(' ', '')[1:]) > 0.81:
            all_ru_regions.iloc[i, -1] = kad_val['dst']
            cnter += 1
        else:
            pass
    if cnter > 2:
        print(cnter, i)

3 511
3 2330
3 2333
3 2342


## check results

In [64]:
all_kads_df[all_kads_df.reg == 66]

,name,reg,dst
223,Кушвинский район,66,53
224,Верхнетуринский район,66,38
225,Карпинский район,66,47
226,Заречный район,66,42
227,Арамильский район,66,33
...,...,...,...
291,Верхнесалдинский район,66,8
292,Артемовский район,66,2
293,Пышминский район,66,20
294,Тугулымский район,66,29


In [65]:
all_ru_regions[(all_ru_regions.dst.isna())]

,name,name__ru,wikipedia,geometry,area,reg,dst
5,Кижингинский район,Кижингинский район,ru:Кижингинский район,"POLYGON ((1054128.056 2335908.509, 1053859.626...",6.315684e+09,None,None
9,Фокино,NaN,ru:Фокино (Брянская область),"POLYGON ((-3422044.060 3631955.613, -3422527.8...",8.495758e+06,32,None
17,Кандалакшский район,Кандалакшский район,ru:Кандалакшский район,"POLYGON ((-2547315.726 4725005.760, -2547963.2...",1.509560e+10,None,None
20,Ирафский район,Ирафский район,ru:Ирафский район,"POLYGON ((-3529005.872 2414510.467, -3528685.2...",1.126943e+09,None,None
23,ЗАТО Заозёрск,NaN,NaN,"POLYGON ((-2348377.495 5003070.991, -2352510.0...",6.903000e+08,None,None
...,...,...,...,...,...,...,...
2311,Эхирит-Булагатский район,NaN,ru:Эхирит-Булагатский район,"POLYGON ((658966.011 2342632.890, 658589.138 2...",4.417625e+09,38,None
2313,Осинский район,NaN,ru:Осинский район (Иркутская область),"POLYGON ((597633.559 2410220.534, 597614.823 2...",3.781590e+09,38,None
2314,Боханский район,NaN,ru:Боханский район,"POLYGON ((596885.970 2404256.684, 596886.641 2...",3.117788e+09,38,None
2332,Нижний Тагил,NaN,ru:Городской округ «город Нижний Тагил»,"POLYGON ((-1890661.511 3234120.118, -1890900.4...",3.933308e+09,66,None


## save

In [66]:
all_ru_regions

,name,name__ru,wikipedia,geometry,area,reg,dst
0,Селенгинский район,Селенгинский район,ru:Селенгинский район,"POLYGON ((772911.849 2183926.259, 770829.137 2...",6.729938e+09,3,18
1,Хоринский район,Хоринский район,ru:Хоринский район,"POLYGON ((975660.756 2392376.746, 968958.946 2...",1.216154e+10,3,21
2,Курумканский район,Курумканский район,ru:Курумканский район,"POLYGON ((1006551.636 2652187.027, 1001762.061...",1.117060e+10,3,11
3,Брюховецкий район,Брюховецкий район,ru:Брюховецкий район,"POLYGON ((-3642267.062 2820599.306, -3642492.8...",1.132083e+09,23,4
4,Павловский район,Павловский район,ru:Павловский район (Краснодарский край),"POLYGON ((-3583010.429 2801858.794, -3583399.7...",1.472633e+09,23,24
...,...,...,...,...,...,...,...
2360,Тамбов,NaN,ru:Тамбов,"POLYGON ((-3124307.505 3303042.861, -3124603.5...",7.601746e+07,68,20
2361,Рассказовский район,NaN,ru:Рассказовский район,"POLYGON ((-3081265.438 3272682.587, -3081149.2...",1.554211e+09,68,15
2362,Сосновский район,NaN,ru:Сосновский район (Тамбовская область),"POLYGON ((-3079943.869 3330265.964, -3082502.7...",2.043358e+09,68,18
2363,Тамбовский район,NaN,ru:Тамбовский район (Тамбовская область),"POLYGON ((-3101331.947 3297711.561, -3101893.6...",2.251141e+09,68,20


In [67]:
all_ru_regions.to_csv('map_data/ru_regions_kadastr.csv', index=0)